# Loss Function: 

La Loss Function o Cost Function es la funcion que elijamos para calcular cómo está performando nuestro modelo. Esta función mapea los valores de las variables del modelo con un número real que representa el "costo" que esa version del modelo tiene respecto a los valores desables. Idealmente, el objetivo del modelo es optimizar el valor de los weights and biases hasta obtener el costo más bajo, para ello. 

Intuitivamente diría que esta función se utiliza con modelos supervisados porque entre sus parametros recibe los targets o labels del modelo, pero esto habría que revisarlo mejor.

En la bibliografía e incluso en pytorch, esta función recibe distintos nombre:

- error function
- criterion function
- cost function
- objective function
- loss function

Si bien, en casos particulares todas la denominaciones hacen referencia a lo mismo, esto no es estrictamente así visto en detalle.

Una diferencia entre funciones cost, loss y objective esta en este [comentario de stack exchange](https://stats.stackexchange.com/questions/179026/objective-function-cost-function-loss-function-are-they-the-same-thing)

La noción de 'criterion' y la idea general de función loss que viene del decision theory se me escapa bastante.


-------------------------------------------------
<sub>To do: explicar estas diferencias de denominación</sub>

## Por qué funciona?

Ok, pero cómo saber qué cambios debemos hacer en nuestros w&b que se inicializaron con valores random, para obtener el resultado más bajo de nuestra función loss? 

Dejemos que alguien lo explique mejor, como, por ejemplo, este [post de kaggle](https://www.kaggle.com/erdemuysal/linear-regression-from-scratch-and-with-pytorch)

"The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

<img src="LOSSIMAGE.png">


The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model."


# Distintas Loss Functions y su aplicación

## Problemas de regresión 

Para problemas de regresión[<sup>1</sup>](#fn1), una de las funciones loss más usadas es el Mean Squared Error, pero también pueden aplicarse  Mean Absolute Error u otras y su elección depende de la distribución de los datos, la presencia de outliers, factores que habría que analizar al momento de tener un corpus.


### Mean Squared Error Loss

Esta función toma los valores esperados (y) y los valores de la prediccion (y_hat) y suma el total de sus distancias cuadradas. El total de esta suma es promediado por el numero de ejemplos.


<img src="MSEL.png">

----------------------------------------------------------------------------------------------------------------
<sub>[<sup id="fn1">1</sup>](#fn1-back)Un problema de regresión es aquel cuyas variables de salida (o outputs) son valores reales o continuos, como 'salario', 'peso', 'costo de una propiedad'.</span></sub>


In [1]:
#Implementación en Pytorch

import torch
import torch.nn as nn

mse_loss = nn.MSELoss()
outputs = torch.randn(3, 5, requires_grad=True)
targets = torch.randn(3, 5)
loss = mse_loss(outputs, targets)
print('output: ', outputs)
print('targets: ', targets)
print('-------------------')
#print('loss: ', loss)

output:  tensor([[ 0.4886,  0.0048, -1.2792, -0.8813,  1.5077],
        [ 0.4006,  0.4693, -1.1326, -0.7136,  1.4769],
        [ 0.4852, -0.1870, -1.1527,  0.2586, -0.6437]], requires_grad=True)
targets:  tensor([[ 1.1903,  0.7374,  0.3875,  0.2201,  0.0893],
        [ 1.1994,  0.0387, -0.5466,  0.6031,  1.2106],
        [ 0.3112, -0.1751,  0.6916,  1.9733, -0.2962]])
-------------------


In [2]:
# Cuenta propia para ver cómo funciona

sum_se = torch.zeros(5)
print('sum_se', sum_se)
for i in zip(targets, outputs):
    se = (i[0] - i[1]) ** 2
    print('se', se)
    sum_se += se
    print('sum_se', sum_se)
print('end', sum_se)
    
sum_num = torch.sum(sum_se)
print(sum_num)

mse = sum_num / 15
#mse = sum_num / len(targets) descomentar para ver el resultado de interpretar N como len de targets en vez de MxN
print(mse)

sum_se tensor([0., 0., 0., 0., 0.])
se tensor([0.4924, 0.5367, 2.7781, 1.2131, 2.0119], grad_fn=<PowBackward0>)
sum_se tensor([0.4924, 0.5367, 2.7781, 1.2131, 2.0119], grad_fn=<AddBackward0>)
se tensor([0.6380, 0.1854, 0.3433, 1.7337, 0.0709], grad_fn=<PowBackward0>)
sum_se tensor([1.1304, 0.7221, 3.1214, 2.9468, 2.0828], grad_fn=<AddBackward0>)
se tensor([3.0256e-02, 1.4266e-04, 3.4015e+00, 2.9402e+00, 1.2071e-01],
       grad_fn=<PowBackward0>)
sum_se tensor([1.1607, 0.7223, 6.5229, 5.8870, 2.2035], grad_fn=<AddBackward0>)
end tensor([1.1607, 0.7223, 6.5229, 5.8870, 2.2035], grad_fn=<AddBackward0>)
tensor(16.4964, grad_fn=<SumBackward0>)
tensor(1.0998, grad_fn=<DivBackward0>)


### Por qué 15?

Cuando tomé la primera descripción de la función 'no me daban los números' y mi resultado no era el mismo que el de aplicar la función de pytorch, hasta que encontré esta descripción que hace explicito que los valores involucrados no son puntos sino matrices mxn.

<img src="MSELMxN.png">

### Linear Regression with california housing dataset (default available in colab)

Con este ejemplo quise ilustrar de manera simple el uso de MSE en un modelo entrenado con pytorch. De ningún modo este modelo funciona, ni en este ni en ningún otro data set de regresión, pero se podria acomodar para que lo haga, empezando por agregar un DataLoader.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [4]:
LR = 0.01 
EPOCHS = 3

In [5]:
train_df = pd.read_csv('./california_housing_train.csv')
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [6]:
#clean empty values and inf
train_df.replace([np.inf, -np.inf], np.nan)  # Replace inf values with NaNs
train_df.dropna(inplace=True)  # Drop NaN values from the dataframe

In [7]:
#inputs, targets
# Convert from Pandas dataframe to numpy arrays
inputs = train_df.drop(['median_house_value'], axis=1).to_numpy()  # Drop TARGET_COLUMN since it is target, can not be used as input
targets = train_df[['median_house_value']].to_numpy()
inputs.shape, targets.shape

((17000, 8), (17000, 1))

In [8]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [9]:
class linearRegression(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        out = self.linear(x)
        return out

In [10]:
model = linearRegression(8, 1) #in_features = 8dim, out_features=1dim

In [11]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [12]:
for epoch in range(EPOCHS):
    

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    print('input: ', inputs[0])
    outputs = model(inputs)
    print('output: ', outputs[0])
    print('target: ', targets[0])

    # get loss for the predicted output
    loss = criterion(outputs, targets)
    #print(loss)
        
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('---------------------------------------------')

input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([-755.7535], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 0, loss 56654794752.0
---------------------------------------------
input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([7.5477e+10], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 1, loss 2.8303811984632096e+21
---------------------------------------------
input:  tensor([-1.1431e+02,  3.4190e+01,  1.5000e+01,  5.6120e+03,  1.2830e+03,
         1.0150e+03,  4.7200e+02,  1.4936e+00])
output:  tensor([-2.3554e+16], grad_fn=<SelectBackward>)
target:  tensor([66900.])
epoch 2, loss 2.773957857786353e+32
---------------------------------------------


## Cross-Entropy Loss 

Para problemas de clasificación vamos a utilizar Cross-Entropy Loss. Cross-entropy tambien es un concepto de proveniente de information theory que tampoco podría explicar, pero esta disponible [aquí](https://en.wikipedia.org/wiki/Cross_entropy).

Dependiendo del problema de clasificación que tengamos entre mano, vamos a usar distintos tipos de cross-entropy, o más bien, vamos a aplicar distintas funciones de activación a nuestro vector de output.


<img src="images.png">


Para problemas multi-clase (a qué intent pertenece una oración, por ejemplo), vamos a aplicar una función Softmax que convierta todos los valores de nuestro vector output en valores entre 0 y 1 de modo que sumen 1 entre sí. Así, cada elemento del output vector puede ser interpretado como la probabilidad de una clase.

Para problemas de clasificación binaria o multi-label, la función de activación será la Sigmoid Function que también lleva todos los números a valores entre 0 y 1 pero sin que entre ellos sumen 1. Si pensamos en un problema con dos clases: el ejemplo recibirá un valor más cercano a 1 para la clase a la que pertenece y más cercano a 0 para la clase a la que no pertenece. Y lo mismo sucede para cada una de las clases a las que pertenece el ejemplo (multi-intent)


-------------------------------------------------
<sub>[Este post sobre cross-entropy](https://gombru.github.io/2018/05/23/cross_entropy_loss/)</sub>


### Categorical Cross-Entropy Loss 

Categorical-cross Entropy loss es usualmente aplicado en problemas de clasificación multi clase donde el output del modelo es interpretado como la probabilidad de que un dato pertenezca a una clase.

En estos problemas vamos a tener outputs reunidos en un vector de salida de tamaño C (número de clases) y el ground truth va a ser un one-hot vector con 1 posición (clase) positiva y C-1 posiciones negativas. 

Como vimos, primero aplica la función Softmax y luego la cross-entropy loss.

<img src="softmax.png">

En la función Softmax el resultado que para cada clase z_i depende de las demás clases (z_j)

El ejemplo de [Categorical Cross Entropy](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) que vamos a ver es nativo de pytorch y está descripto como:

'This criterion combines LogSoftmax and NLLLoss in one single class.'

La relación entre cross-entropy y NLLLoss (Negative Log-Likelihood) está también descripta en el artículo de Wikipedia sobre cross-entropy en information theory y tampoco puedo explicarlo ahora.



In [34]:
import torch
import torch.nn as nn

ce_loss = nn.CrossEntropyLoss()
outputs = torch.randn(3, 5, requires_grad=True)
targets = torch.tensor([1, 0, 3], dtype=torch.int64)
loss = ce_loss(outputs, targets)
print(loss)

tensor(2.3808, grad_fn=<NllLossBackward>)


In [35]:
print(outputs[0])
print(targets[0])

tensor([0.4656, 0.6432, 0.3897, 0.2335, 0.8412], grad_fn=<SelectBackward>)
tensor(1)


In [36]:
#To-do: cuenta manual
# Cuenta propia para ver cómo funciona

### Clasificación Multi Clase con MNIST dataset (default available in colab)

In [17]:
train_df = pd.read_csv('./mnist_train_small.csv', header=None, index_col=None)
print(train_df.head())
print('length columns: ', len(train_df.columns)) #28px x 28px = 784 + label = 1 --> 785 columns
train_df.columns[0], train_df.columns[300], train_df.columns[784] #??

   0    1    2    3    4    5    6    7    8    9    ...  775  776  777  778  \
0    6    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1    5    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2    7    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    9    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    5    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   779  780  781  782  783  784  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 785 columns]
length columns:  785


(0, 300, 784)

In [19]:
#inputs, labels
# Convert from Pandas dataframe to numpy arrays
inputs = train_df.drop([0], axis=1).to_numpy()  
labels = train_df[0].to_numpy()
inputs.shape, labels.shape

((20000, 784), (20000,))

In [20]:
labels[0], labels[18], labels[1880]

(6, 7, 8)

In [21]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
labels = torch.from_numpy(labels)
labels = labels.type(torch.LongTensor) #la función cross-entropy espera un valor representando la categoria

In [22]:
class Classification(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Classification, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        out = self.linear(x)
        return out

In [23]:
model = Classification(784,10)

In [27]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [33]:
for epoch in range(EPOCHS):
    

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    #print('input: ', inputs[0])
    outputs = model(inputs)
    print('labels: ', labels[0])
    #print('output: ', loss_outputs)
    # get loss for the predicted output
    loss = criterion(outputs, labels)
        
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))
    print('---------------------------------------------')

labels:  tensor(6)
tensor(1720.8148, grad_fn=<NllLossBackward>)
epoch 0, loss 1720.8148193359375
---------------------------------------------
labels:  tensor(6)
tensor(1122.7852, grad_fn=<NllLossBackward>)
epoch 1, loss 1122.78515625
---------------------------------------------
labels:  tensor(6)
tensor(886.0213, grad_fn=<NllLossBackward>)
epoch 2, loss 886.0213012695312
---------------------------------------------


### Binary Cross-Entropy Loss (binary classification, and multilabel)

En los problemas de multi-labelling, cada muestra puede pertenecer a más de una clase. Es decir que el vector target ya no va a ser un one-hot vector, si no un vector de 0s y 1s con dimension C. Estos problemas se reformulan como C clasificaciones binarias independientes entre sí. Por esto, la BCELoss es tambien la función que utilizamos para clasificaciones binarias (como sentiment analysis)

En este caso aplicamos primero la función de activación Sigmoid. Esta función se aplica independientemente a cada clase. También se la llama función logística. 


<img src="sigmoid.png">

Pytorch tiene dos implementaciones: [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) y [BCEWITHLOGITSLOSS](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)

La primera solo aplica la función loss mientras que la segunda combina la capa de función Sigmoid con la capa de loss. Notar cómo en el siguiente ejemplo se aplica la BCELoss y por eso, antes se calcula la sigmoidal de los outputs.

In [37]:
bce_loss = nn.BCELoss()
sigmoid = nn.Sigmoid()
probabilities = sigmoid(torch.randn(4, 1, requires_grad=True))
targets = torch.tensor([1, 0, 1, 0],  dtype=torch.float32).view(4, 1)
loss = bce_loss(probabilities, targets)
print(probabilities)
print(loss)


tensor([[0.8484],
        [0.5441],
        [0.2829],
        [0.5932]], grad_fn=<SigmoidBackward>)
tensor(0.7780, grad_fn=<BinaryCrossEntropyBackward>)


In [38]:
#To-do: cuenta manual
# Cuenta propia para ver cómo funciona

In [39]:
#To-do: ejemplo canónico de problema de claificación binaria.

### Negative Log Likelihood for CRF



### Combining Loss Functions for Multi Task Learning problems


<img src="Diet.png">